# While Loop with AiiDA WorkGraph

This notebook demonstrates using the WhileNode with AiiDA WorkGraph backend.

## Setup

In [ ]:
import sys
from pathlib import Path

# Add source to path
src_path = Path.cwd().parent / "python_workflow_definition" / "src"
sys.path.insert(0, str(src_path))

# Add workflow examples to path so they can be imported
workflow_examples_path = Path.cwd().parent / "example_workflows" / "while_loop"
sys.path.insert(0, str(workflow_examples_path))

from aiida import load_profile
load_profile()

## Example 1: Load While Loop from JSON

Load a simple counter workflow with while loop from JSON.

In [2]:
from python_workflow_definition.aiida import load_workflow_json

# Create the JSON workflow file path
json_file = Path.cwd().parent / "example_workflows" / "while_loop" / "simple_counter.json"

# Load the workflow
wg = load_workflow_json(str(json_file))

print(f"Loaded WorkGraph with {len(wg.tasks)} tasks")
print(f"Tasks: {[t.name for t in wg.tasks]}")

ModuleNotFoundError: No module named 'workflow'

## Example 2: Build WorkGraph and Run

Build and execute the while loop workflow.

In [ ]:
# Submit the workflow
wg.submit(wait=True)

print(f"Workflow state: {wg.state}")
print(f"Result: {wg.tasks[-1].outputs}")

## Example 3: Expression-based Condition

Load a while loop with expression-based condition.

In [ ]:
json_file_expr = Path.cwd().parent / "example_workflows" / "while_loop" / "simple_counter_expression.json"

wg_expr = load_workflow_json(str(json_file_expr))
wg_expr.submit(wait=True)

print(f"Expression-based workflow state: {wg_expr.state}")

## Example 4: Create While Loop Manually

Create a while loop workflow using Python API and export to JSON.

In [ ]:
from python_workflow_definition.models import (
    PythonWorkflowDefinitionWorkflow,
    PythonWorkflowDefinitionWhileNode,
    PythonWorkflowDefinitionInputNode,
    PythonWorkflowDefinitionOutputNode,
    PythonWorkflowDefinitionEdge,
)

# Define workflow
workflow = PythonWorkflowDefinitionWorkflow(
    version="0.1.0",
    nodes=[
        PythonWorkflowDefinitionInputNode(id=0, type="input", name="n", value=5),
        PythonWorkflowDefinitionInputNode(id=1, type="input", name="m", value=0),
        PythonWorkflowDefinitionWhileNode(
            id=2,
            type="while",
            conditionExpression="m < n",
            bodyFunction="workflow.increment_m",
            maxIterations=100,
        ),
        PythonWorkflowDefinitionOutputNode(id=3, type="output", name="result"),
    ],
    edges=[
        PythonWorkflowDefinitionEdge(source=0, target=2, targetPort="n"),
        PythonWorkflowDefinitionEdge(source=1, target=2, targetPort="m"),
        PythonWorkflowDefinitionEdge(source=2, sourcePort="m", target=3),
    ],
)

# Save to JSON
output_file = Path("/tmp/custom_while_loop.json")
workflow.dump_json_file(output_file)

print(f"Saved workflow to {output_file}")
print(workflow.dump_json())

## Example 5: Load and Execute Custom Workflow

In [ ]:
# Load the custom workflow
wg_custom = load_workflow_json(str(output_file))
wg_custom.submit(wait=True)

print(f"Custom workflow completed with state: {wg_custom.state}")

## Summary

This notebook demonstrated:
1. Loading while loop workflows from JSON
2. Executing workflows with AiiDA WorkGraph
3. Using both function-based and expression-based conditions
4. Creating workflows programmatically with the Python API
5. Exporting workflows to JSON format